In [ ]:
% matplotlib inline 

import numpy as np 

import pygwb.delta_sigma_cut
import pygwb.baseline

In [ ]:
flow = 50
fhigh = 500
segment_duration = 192  # also fftlength in pre-processing
overlap = 0.5
segment_duration = 192  # also fftlength in pre-processing
sampling_frequency = 4096
dsc = 0.2
alphas = [-5, 0, 3]
notch_file = ""

In [ ]:
pickled_base = baseline.Baseline.load_from_pickle(
    "test/test_data/H1L1_1247644138-1247645038.pickle"
)
pickled_ifo_1 = pickled_base.interferometer_1
pickled_ifo_2 = pickled_base.interferometer_2
naive_psd_1 = pickled_ifo_1.psd_spectrogram
naive_psd_2 = pickled_ifo_2.psd_spectrogram
avg_psd_1 = pickled_ifo_1.average_psd
avg_psd_2 = pickled_ifo_2.average_psd

dF = avg_psd_1.frequencies.value[1] - avg_psd_1.frequencies.value[0]
naive_psd_1 = naive_psd_1.crop_frequencies(flow, fhigh + dF)
naive_psd_2 = naive_psd_2.crop_frequencies(flow, fhigh + dF)
avg_psd_1 = avg_psd_1.crop_frequencies(flow, fhigh + dF)
avg_psd_2 = avg_psd_2.crop_frequencies(flow, fhigh + dF)

# calculate CSD
stride = segment_duration - overlap
csd_segment_offset = int(np.ceil(segment_duration / stride))

# also remove naive psds from edge segments
naive_psd_1 = naive_psd_1[csd_segment_offset : -(csd_segment_offset + 1) + 1]
naive_psd_2 = naive_psd_2[csd_segment_offset : -(csd_segment_offset + 1) + 1]

In [ ]:
badGPStimes, _ = delta_sigma_cut.run_dsc(
    dsc=dsc,
    segment_duration=segment_duration,
    sampling_frequency=sampling_frequency,
    psd1_naive=naive_psd_1,
    psd2_naive=naive_psd_2,
    psd1_slide=avg_psd_1,
    psd2_slide=avg_psd_2,
    alphas=alphas,
    orf=np.array([1]),
    notch_list_path=notch_file,
)